In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow import keras

In [3]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

In [4]:
# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

In [5]:
# 입력층 데이터의 모양(shape)
print(x_train_reshaped.shape)

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다.
X = x_train_reshaped[:5]

(60000, 784)


In [6]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
weight_init_std = 0.1
input_size = 784
hidden_size=50

In [8]:
# 인접 레이어간 관계를 나타내는 파라미터 W를 생성하고 random 초기화 ? 
W1 = weight_init_std * np.random.randn(input_size, hidden_size)  

In [11]:
# 바이어스 파라미터 b를 생성하고 Zero로 초기화
b1 = np.zeros(hidden_size)

In [13]:
a1 = np.dot(X, W1) + b1   # 은닉층 출력 , dot : 행렬곱 수행

In [14]:
print('W1.shape:{} : 인접 레이어간 관계 나타내는 파라미터'.format(W1.shape))
print('X.shape:{}  : 인풋 값'.format(X.shape))
print('b1.shape:{} : bias parameter'.format(b1.shape))
print('a1.shape:{} : (은닉층)'.format(a1.shape))

W1.shape:(784, 50) : 인접 레이어간 관계 나타내는 파라미터
X.shape:(5, 784)  : 인풋 값
b1.shape:(50,) : bias parameter
a1.shape:(5, 50) : (은닉층)


In [35]:
# Numpy
# ext : exp 함수는 자연 상수 e의 x 제곱 값을 계산해주는 함수입니다. e는 약 2.71828로, 수학적으로 중요한 상수입니다. x 값에 따라 결과값이 크게 변합니다. x가 양수인 경우 결과값이 점점 커지고, 음수인 경우 결과값이 점점 작아집니다.

예를 들어, exp(1)은 e의 1제곱을 계산하여 약 2.71828이 됩니다. 또한, exp(-1)은 e의 -1제곱을 계산하여 약 0.36788이 됩니다. exp 함수는 주로 확률 계산, 지수 함수와 관련된 수식, 머신러닝 등에서 사용됩니다.
# 위 수식의 sigmoid 함수를 구현해 봅니다.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  


z1 = sigmoid(a1)
print(z1[0])  # sigmoid의 출력은 모든 element가 0에서 1사이

[0.40969909 0.4780367  0.55549474 0.47533048 0.1226221  0.57734689
 0.32620675 0.46843504 0.62801602 0.51026387 0.48359554 0.47160468
 0.60402354 0.67258598 0.70872806 0.3517376  0.30344372 0.6701429
 0.58602843 0.36882563 0.71878955 0.50230475 0.27902201 0.79458623
 0.64137042 0.83653965 0.06388716 0.30146292 0.59702953 0.93512102
 0.45715317 0.38208308 0.8075804  0.46605151 0.50517457 0.54271896
 0.44856744 0.42127238 0.26071155 0.83811965 0.83402485 0.46258909
 0.33592118 0.38980843 0.27522965 0.30401636 0.62350722 0.17254492
 0.57924997 0.55469875]


In [36]:
# 단일 레이어 구현 함수
def affine_layer_forward(X, W, b):
    y = np.dot(X, W) + b
    cache = (X, W, b)
    return y, cache

In [37]:
input_size = 784
hidden_size = 50
output_size = 10

W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
a2, cache2 = affine_layer_forward(z1, W2, b2)    # z1이 다시 두번째 레이어의 입력이 됩니다. 

print(a2[0])  # 최종 출력이 output_size만큼의 벡터가 되었습니다.

[ 0.23197957 -0.16533228 -0.01257295 -0.01980828 -0.16725585  0.16754595
  0.08031149 -0.13923419  0.21737013  0.46333025]


In [39]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [40]:
y_hat = softmax(a2)
y_hat[0]  # 10개의 숫자 중 하나일 확률이 되었습니다.

array([0.11579451, 0.07782832, 0.09067346, 0.09001977, 0.07767875,
       0.10856874, 0.09949915, 0.07988622, 0.11411512, 0.14593596])

In [41]:
# 정답 라벨을 One-hot 인코딩하는 함수
def _change_one_hot_label(X, num_category):
    T = np.zeros((X.size, num_category))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T

Y_digit = y_train[:5]
t = _change_one_hot_label(Y_digit, 10)
t     # 정답 라벨의 One-hot 인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [42]:
print(y_hat[0])
print(t[0])

[0.11579451 0.07782832 0.09067346 0.09001977 0.07767875 0.10856874
 0.09949915 0.07988622 0.11411512 0.14593596]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [43]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

Loss = cross_entropy_error(y_hat, t)
Loss

2.2234837053114536

X-Y 좌표축의 기울기란 X의 변화에 따른 Y의 변화량을 의미합니다.
Y를 X로 미분해서 구하지요.

우리는 파라미터 W의 변화에 따른 오차(Loss) L의 변화량을 구하려고 합니다. 
그러면 오차 기울기가 커지는 방향의 반대 방향으로 파라미터를 조정해 주면 됩니다. 
단, 조정을 너무 많이 해주면 안 되기 때문에 적절한 step size 역할을 하는 learning rate가 필수적입니다.
그 과정을 Numpy를 통해 구현해 보겠습니다



In [44]:
batch_num = y_hat.shape[0]
dy = (y_hat - t) / batch_num
dy    # softmax값의 출력으로 Loss를 미분한 값

array([[ 0.0231589 ,  0.01556566,  0.01813469,  0.01800395,  0.01553575,
        -0.17828625,  0.01989983,  0.01597724,  0.02282302,  0.02918719],
       [-0.1744187 ,  0.01647559,  0.020092  ,  0.01676362,  0.01708868,
         0.01827082,  0.02384309,  0.0165788 ,  0.02110274,  0.02420336],
       [ 0.01956854,  0.01695509,  0.01609279,  0.01844714, -0.18579513,
         0.02254229,  0.02344842,  0.01330711,  0.02512853,  0.03030522],
       [ 0.01860023, -0.18292094,  0.01726501,  0.01913076,  0.01434541,
         0.02125701,  0.02312842,  0.01288163,  0.02468118,  0.03163129],
       [ 0.0219918 ,  0.017583  ,  0.0153116 ,  0.01845695,  0.01319414,
         0.02020265,  0.02148487,  0.01380497,  0.02270388, -0.16473387]])

In [50]:
dW2 = np.dot(z1.T, dy)    
#dw2

In [46]:
dW2 = np.dot(z1.T, dy)
db2 = np.sum(dy, axis=0)

In [47]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [48]:
dz1 = np.dot(dy, W2.T)
da1 = sigmoid_grad(a1) * dz1
dW1 = np.dot(X.T, da1)
db1 = np.sum(dz1, axis=0)

In [49]:
learning_rate = 0.1

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    return W1, b1, W2, b2

backPropagation

In [51]:
def affine_layer_backward(dy, cache):
    X, W, b = cache
    dX = np.dot(dy, W.T)
    dW = np.dot(X.T, dy)
    db = np.sum(dy, axis=0)
    return dX, dW, db

In [52]:
# 파라미터 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

# Forward Propagation
a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
a2, cache2 = affine_layer_forward(z1, W2, b2)

# 추론과 오차(Loss) 계산
y_hat = softmax(a2)
t = _change_one_hot_label(Y_digit, 10)   # 정답 One-hot 인코딩
Loss = cross_entropy_error(y_hat, t)

print(y_hat)
print(t)
print('Loss: ', Loss)
        
dy = (y_hat - t) / X.shape[0]
dz1, dW2, db2 = affine_layer_backward(dy, cache2)
da1 = sigmoid_grad(a1) * dz1
dX, dW1, db1 = affine_layer_backward(da1, cache1)

# 경사하강법을 통한 파라미터 업데이트    
learning_rate = 0.1
W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

[[0.06576426 0.09035934 0.12743167 0.0724405  0.13635358 0.10359503
  0.11730042 0.1000583  0.12420202 0.06249489]
 [0.07606983 0.08528226 0.10845392 0.06604837 0.13474525 0.11003996
  0.12495271 0.10730389 0.12496396 0.06213985]
 [0.07022802 0.08928499 0.09778169 0.08113852 0.10470959 0.11637719
  0.09376612 0.12171368 0.15039375 0.07460644]
 [0.06256311 0.07210534 0.12760216 0.06289305 0.15901344 0.10640032
  0.10702223 0.12239998 0.12202856 0.05797182]
 [0.05595216 0.07967916 0.13013205 0.08354202 0.15001531 0.11504139
  0.09942496 0.10214236 0.11995343 0.06411716]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Loss:  2.495320917164009


In [53]:
W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

def train_step(X, Y, W1, b1, W2, b2, learning_rate=0.1, verbose=False):
    a1, cache1 = affine_layer_forward(X, W1, b1)
    z1 = sigmoid(a1)
    a2, cache2 = affine_layer_forward(z1, W2, b2)
    y_hat = softmax(a2)
    t = _change_one_hot_label(Y, 10)
    Loss = cross_entropy_error(y_hat, t)

    if verbose:
        print('---------')
        print(y_hat)
        print(t)
        print('Loss: ', Loss)
        
    dy = (y_hat - t) / X.shape[0]
    dz1, dW2, db2 = affine_layer_backward(dy, cache2)
    da1 = sigmoid_grad(a1) * dz1
    dX, dW1, db1 = affine_layer_backward(da1, cache1)
    
    W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    
    return W1, b1, W2, b2, Loss

In [54]:
X = x_train_reshaped[:5]
Y = y_train[:5]

# train_step을 다섯 번 반복 돌립니다.
for i in range(5):
    W1, b1, W2, b2, _ = train_step(X, Y, W1, b1, W2, b2, learning_rate=0.1, verbose=True)

---------
[[0.09236903 0.09657525 0.07634385 0.07900484 0.13660977 0.07588111
  0.12569022 0.18092852 0.05372159 0.08287582]
 [0.08006172 0.10415488 0.06237974 0.07930751 0.15039766 0.0856675
  0.11164213 0.18692942 0.05945226 0.08000717]
 [0.08395383 0.10813766 0.06291858 0.0772277  0.15298903 0.06783926
  0.11507906 0.18823689 0.06818133 0.07543665]
 [0.06957422 0.09634504 0.05978168 0.07898502 0.14186913 0.08026622
  0.12159811 0.19284597 0.06528002 0.09345458]
 [0.06646181 0.10609495 0.04970261 0.06562071 0.15846614 0.07175694
  0.12494733 0.22221849 0.0661137  0.06861731]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Loss:  2.3999927654280206
---------
[[0.10992482 0.10902204 0.07041008 0.07177389 0.14684834 0.09458559
  0.10858691 0.13919244 0.05018888 0.09946701]
 [0.10082826 0.1159406  0.05796885 0.07167243 0.1608966  0.10219133
  0.09534771 0.14262828 0.0546

Accuracy

In [55]:
def predict(W1, b1, W2, b2, X):
    a1 = np.dot(X, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

In [56]:
# X = x_train[:100] 에 대해 모델 추론을 시도합니다. 
X = x_train_reshaped[:100]
Y = y_test[:100]
result = predict(W1, b1, W2, b2, X)
result[0]

array([0.15773553, 0.13170041, 0.04670236, 0.04637105, 0.15112585,
       0.16201234, 0.06311526, 0.06744765, 0.03518633, 0.13860322])

In [57]:
def accuracy(W1, b1, W2, b2, x, y):
    y_hat = predict(W1, b1, W2, b2, x)
    y_hat = np.argmax(y_hat, axis=1)

    accuracy = np.sum(y_hat == y) / float(x.shape[0])
    return accuracy

In [58]:
acc = accuracy(W1, b1, W2, b2, X, Y)

t = _change_one_hot_label(Y, 10)
print(result[0])
print(t[0])
print(acc)

[0.15773553 0.13170041 0.04670236 0.04637105 0.15112585 0.16201234
 0.06311526 0.06744765 0.03518633 0.13860322]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
0.17
